In [73]:
import joblib
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

juegos = pd.read_csv('databases/juegos_procesados.csv')
libros = pd.read_csv('databases/libros_procesados.csv')
peliculas = pd.read_csv('databases/peliculas_procesados.csv')
sentimientos = pd.read_csv('databases/sentimientos.csv')


In [74]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')


In [ ]:
def predict_sentiment(text):
    # Step 1: Encode the input text to get embeddings
    text_vectorized = embedding_model.encode([text])
    
    # Step 2: Load the logistic regression model
    try:
        modelo_LR = joblib.load('modelo_LR.pkl')
    except:
        print("No se ha encontrado el modelo, correr el notebook de clasificacion")
    
    # Step 3: Predict the sentiment
    sentiment_class = modelo_LR.predict(text_vectorized)[0]
    
    # Step 4: Map the sentiment class to a label
    labels = {0: "Melancolico", 1: "Ni fu Ni fa", 2: "Alegre"}
    sentiment_label = labels[sentiment_class]
    
    return sentiment_label

In [ ]:

# Define text areas and buttons
initial_prompt = widgets.Textarea(
    description='Predictor:',
    value='Hola, ¿Cómo estás?',
    layout=widgets.Layout(width='100%'),
    disabled=True
)

followup_prompt = widgets.Textarea(
    description='Predictor:',
    value='Parece que te estás sintiendo reemplazarPorSentimiento. ¿Qué temas te gustaría explorar?',
    disabled=True,
    layout=widgets.Layout(width='100%')
)

user_initial_input = widgets.Text(
    placeholder='Escriba una respuesta',
    layout=widgets.Layout(width='100%'),
    disabled=False
)

user_followup_input = widgets.Text(
    placeholder='Escriba una respuesta',
    layout=widgets.Layout(width='100%'),
    disabled=False
)

button_initial = widgets.Button(description='Responder')
button_followup = widgets.Button(description='Responder')
output = widgets.Output()

recommendation_display = widgets.Textarea(
    description='Predictor:',
    layout=widgets.Layout(width='100%'),
    disabled=True
)

# Function to provide a recommendation based on sentiment
def get_recommendation(sentiment):
    pass
    # Todo: Implement recommendation logic based on the followup prompt

# Callback for initial response
def on_initial_button_clicked(b):
    with output:
        button_initial.layout.display = 'none'
        output.clear_output()
        sentiment = predict_sentiment(user_initial_input.value)
        print(f"Tu estado de ánimo es: {sentiment}")
        followup_prompt.value = followup_prompt.value.replace('reemplazarPorSentimiento', sentiment.lower())
        display(followup_prompt)
        display(user_followup_input)
        display(button_followup)

# Callback for follow-up response
def on_followup_button_clicked(b):
    with output:
        button_followup.layout.display = 'none'
        sentiment = predict_sentiment(user_initial_input.value)
        recommendation = get_recommendation(sentiment)
        recommendation_display.value = f"Basado en tu estado de ánimo, te recomendamos: {recommendation}"
        display(recommendation_display)

# Bind buttons to callbacks
button_initial.on_click(on_initial_button_clicked)
button_followup.on_click(on_followup_button_clicked)

# Display initial prompt and input
display(initial_prompt)
display(user_initial_input)
display(button_initial)
display(output)


Textarea(value='Hola, ¿Cómo estás?', description='Predictor:', disabled=True, layout=Layout(width='100%'))

Text(value='', layout=Layout(width='100%'), placeholder='Escriba una respuesta')

Button(description='Responder', style=ButtonStyle())

Output()